This notebook shows how to perform a single inference on:

1. Cloud
2. Locally

### Inference on Cloud

In [ ]:
# To run inference on NTNU cloud
## only for testing purpose


import grpc                    # for cloud inference
#import yolov5_service_pb2      # for cloud inference
#import yolov5_service_pb2_grpc # for cloud inference
import pipeline_pb2_grpc       # for cloud visualization
import pipeline_pb2            # for cloud visualization
from PIL import Image, ImageFile
from functions import image_to_byte_array
from icecream import ic

_YOLO_PORT = 8055
_VIS_PORT = 8056


yolo_channel = grpc.insecure_channel("ai4eu.idi.ntnu.no:" + str(_YOLO_PORT))
yolo_stub = pipeline_pb2_grpc.YoloV5Stub(yolo_channel)

visualization_channel = grpc.insecure_channel("ai4eu.idi.ntnu.no:" + str(_VIS_PORT))
visualization_stub = pipeline_pb2_grpc.VisualizationServiceStub(visualization_channel)


image_path = '../utils/reference-images/jervskogen_1_2021-12-11_11-30-03.png'  # or file, Path, PIL, OpenCV, numpy, list

image = Image.open(image_path)


 # (B) Do inference on cloud and send image... 
request = pipeline_pb2.Image(data = image_to_byte_array(image))
detected_objects = yolo_stub.detect(request) 
#detected_objects = detected_objects.objects

ic(detected_objects)
request = pipeline_pb2.ImageWithObjects(image = pipeline_pb2.Image(data = image_to_byte_array(image)),objects = detected_objects)



visualization_stub.Visualize(request)


### Local Inference

In [6]:
import torch


# To run model on local machine
# Take image, run inference and show results
## Only for testing purpose

# Images
image_path = '../utils/reference-images/jervskogen_1_2021-12-11_11-30-03.png'  # or file, Path, PIL, OpenCV, numpy, list

# Model, downloads model from hub and stores the model file with weights in the same directory
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True, pretrained=True)  # or yolov5m, yolov5l, yolov5x, custom
model.conf = 0.2
# Yolo Inference
results = model(image_path, size=640)

# Results
results.print()  # or .print() .show(), .save(), .crop(), .pandas(), etc.
ic(type(results.pandas()))
ic(results.pandas().xyxyn[0])
results.show()



Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /Users/areeb/.cache/torch/hub/master.zip
fatal: not a git repository (or any of the parent directories): .git
YOLOv5 🚀 2022-3-31 torch 1.10.2 CPU

Fusing layers... 
[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.5 GFLOPs
Adding AutoShape... 
image 1/1: 1080x1920 3 persons
Speed: 65.1ms pre-process, 310.2ms inference, 1.2ms NMS per image at shape (1, 3, 384, 640)
ic| type(results.pandas()): <class 'models.common.Detections'>
ic| results.pandas().xyxyn[0]:        xmin      ymin      xmax      ymax  confidence  class    name
                               0  0.605944  0.130734  0.619040  0.172764    0.382782      0  person
                               1  0.478418  0.090557  0.486682  0.119556    0.244386      0  person
                               2  0.932291  0.559586  0.948298  0.600477    0.205561      0  person


#### Playground

In [ ]:
# The below code is just to test the resizing of an image, will be removed
# Images
im_pth = '../utils/reference-images/jervskogen_1_2021-12-11_11-30-03.png'

desired_size = 640
#im_pth = "/home/jdhao/test.jpg"

im = Image.open(im_pth)
old_size = im.size  # old_size[0] is in (width, height) format

ratio = float(desired_size)/max(old_size)
new_size = tuple([int(x*ratio) for x in old_size])
# use thumbnail() or resize() method to resize the input image

# thumbnail is a in-place operation

# im.thumbnail(new_size, Image.ANTIALIAS)

im = im.resize(new_size, Image.ANTIALIAS)
# create a new image and paste the resized on it

new_im = Image.new("RGB", (desired_size, desired_size))
new_im.paste(im, ((desired_size-new_size[0])//2,
                    (desired_size-new_size[1])//2))
new_im.size

#new_im.show()